<a href="https://colab.research.google.com/github/CalculatedContent/ww-phys_theory/blob/master/Compare_Jacobian_Diag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Compare Diagonal Jacobian on ResNet 1 batch

Why is my data different from Johns ?


In [1]:
!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [0]:
thismodel = 'resnet20_cifar10'
inum_start = 0

In [3]:
!pip install ipython-autotime
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=69abbf5ede05ad7e20bdde740bb4052d6093044317342c8d29b9bd50b70ea8db
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [4]:
!pip install pytorchcv
!pip install powerlaw

     |████████████████████████████████| 440kB 3.5MB/s 
  Created wheel for powerlaw: filename=powerlaw-1.4.6-cp36-none-any.whl size=24787 sha256=34e7d52b8f17424f66b2e97113d8fdaf8671cb7e7632bda0cdb056d61f81940c
  Stored in directory: /root/.cache/pip/wheels/e0/27/02/08d0e2865072bfd8d7c655e290521e3feca3fc22f1ac460601
Successfully built powerlaw
time: 7.58 s


In [5]:
!pip install GPUtil
import GPUtil

GPUtil.showUtilization()

  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=04170e56c175a64897bb16375b52027c145ba4d51ff0f23f1ae21bd39c1c0276
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
time: 4.03 s


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
time: 6min 37s


In [13]:
Jdir =  '/content/drive/My Drive/test_jacobians'

time: 1.17 ms


In [14]:
#!mkdir "$Jdir"
#!ls "$Jdir"

time: 876 µs


In [15]:
!cd "$Jdir";pwd

/bin/bash: line 0: cd: /content/drive/My Drive/test_jacobians: No such file or directory
/content
time: 1.02 s


In [16]:
!import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import math
import copy


/bin/bash: import: command not found
time: 771 ms


In [51]:
def get_data(batch_size=100, train_range=None, test_range=None):
  normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]], std=[x / 255.0 for x in [63.0, 62.1, 66.7]])
  transform_train = transforms.Compose([
          transforms.ToTensor(),
          normalize])
  transform_test = transforms.Compose([
        transforms.ToTensor(),
        normalize])
  train_dataset = datasets.CIFAR10(root='data', 
                                train=True, 
                                transform=transform_train,
                                download=True)
  test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transform_test,
                                download=True)  
  
  if train_range:
      train_dataset = torch.utils.data.Subset(train_dataset, train_range)

  if test_range:
      teat_dataset = torch.utils.data.Subset(test_dataset, test_range)


  train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size,
                          num_workers=1,
                          shuffle=False)
  test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         num_workers=1,
                         shuffle=False)
  return train_dataset, test_dataset, train_loader, test_loader

time: 14.8 ms


### $J$ Jacobian (replaces native pytorch 1.5 code)

Uses autograd 





In [95]:
from torch.autograd.gradcheck import zero_gradients

def fast_jacobian(inputs, output, batch_size, data_dim):
	"""
	input: input for the function for which the Jacobian will
	computed. It will be batch_size*data_dim. Make sure that the
	input is flagged as requires_grad=True with the torch.autograd.Variable
	wrapper. 
	output: output of the function for which the Jacobian will
	be computed. It will be batch_size*classes
	return: Jacobian of dimension batch_size*classes*data_dim
	"""
	assert inputs.requires_grad

	num_classes = output.size()[1] #0 index is batch

	jacobian = torch.zeros(num_classes, *inputs.size())
	grad_output = torch.zeros(*output.size())
	if inputs.is_cuda:
		grad_output = grad_output.cuda()
		jacobian = jacobian.cuda()

	#zero out gradients
	#compute gradient for one of the classes
	for i in range(num_classes):
		zero_gradients(inputs)
		grad_output.zero_()
		grad_output[:,i] = 1
		output.backward(grad_output, retain_graph=True)
		jacobian[i] = inputs.grad.data

	J =  torch.transpose(jacobian, dim0=0, dim1=1)
	print(J.shape, data_dim)
	J = J.reshape(batch_size, data_dim)
 
	return J


time: 13 ms


### Compare old and new diagonal;


### ResNet 

In [96]:
import pytorchcv
from pytorchcv.model_provider import get_model as ptcv_get_model

for modelname in pytorchcv.model_provider._models.keys():
    if modelname.startswith('resnet') and modelname.endswith('cifar10'):
        print(modelname)

resnet20_cifar10
resnet56_cifar10
resnet110_cifar10
resnet164bn_cifar10
resnet272bn_cifar10
resnet542bn_cifar10
resnet1001_cifar10
resnet1202_cifar10
time: 2.85 ms


In [134]:
num_classes= 10
batch_size = 5
device = 'cuda:0'

num_data = 50000
train_dataset, _, train_loader, _ = get_data(batch_size=batch_size, train_range=range(1000))

for modelname in pytorchcv.model_provider._models.keys():
    if modelname == thismodel:
      print("this model found {}".format(modelname))
      print("num {}. {} - {}".format(0, istart, iend))

      train_dataset, test_dataset, train_loader, test_loader = get_data(batch_size=batch_size)

      model = ptcv_get_model(modelname, pretrained=True)

Files already downloaded and verified
Files already downloaded and verified
this model found resnet20_cifar10
num 0. 0 - 3
Files already downloaded and verified
Files already downloaded and verified
time: 3.7 s


In [135]:
Jd_old, Jd_new = [], []

model = model.to(device)

data_dim = 3*32*32

for batch, data in enumerate(train_loader):
  features, labels = data
  
  print(labels)
  inputs = features.to(device)
  inputs.requires_grad =True
  outputs = model(inputs)
  print("features  ",features.size())

  print("outputs  ",outputs.size())


  Jnew = fast_jacobian(inputs, outputs, batch_size, num_classes*data_dim)
  print(Jnew.shape)
  Jt = Jnew.clone().transpose_(0,1)
  batch_diag = torch.mm(Jnew,Jt).to('cpu')

  for ib in range(batch_size):
    Jd_new.append(batch_diag[ib, ib].to('cpu').numpy())

  features = features.to(device)

  Jold = torch.autograd.functional.jacobian(model, features)
  print(Jold.shape)
  Jold = Jold.view(batch_size,num_classes*batch_size*data_dim)
  print(Jold.shape)

  Jt = Jold.clone().transpose_(0,1)
  batch_diag = torch.mm(Jold,Jt).to('cpu')
  
  del Jt
  torch.cuda.empty_cache()

  for ib in range(batch_size):
    Jd_old.append(batch_diag[ib, ib].to('cpu').numpy())

  del batch_diag
  torch.cuda.empty_cache()

  break


tensor([6, 9, 9, 4, 1])
features   torch.Size([5, 3, 32, 32])
outputs   torch.Size([5, 10])
torch.Size([5, 10, 3, 32, 32]) 30720
torch.Size([5, 30720])
torch.Size([5, 10, 5, 3, 32, 32])
torch.Size([5, 153600])
time: 422 ms


## batch size 1

the old and new Jacobian give the same result

In [133]:
J1old = Jold.cpu().numpy()
J1new = Jnew.cpu().numpy()
np.linalg.norm(J1old), np.linalg.norm(J1new), J1old[0,:],J1new[0,:],

(7.4762306,
 7.47623,
 array([-0.01228475,  0.02412284, -0.02730069, ..., -0.00208849,
        -0.01120334, -0.00289092], dtype=float32),
 array([-0.01228475,  0.02412285, -0.0273007 , ..., -0.00208849,
        -0.01120335, -0.00289093], dtype=float32))

time: 9.55 ms


### batch size 5

the old and new Jacobian give the different results from each other, and from batch size = 1

In [136]:
J5old = Jold.cpu().numpy()
J5new = Jnew.cpu().numpy()
np.linalg.norm(J5old), np.linalg.norm(J5new), J5old[0,:],J5new[0,:],

(65.57009,
 18.058924,
 array([ 0.05852893,  0.00433077, -0.02412006, ..., -0.00774385,
         0.00893848, -0.00247222], dtype=float32),
 array([ 0.00134326,  0.00259299,  0.00795665, ..., -0.00285811,
        -0.00082788,  0.00045752], dtype=float32))

time: 9.05 ms
